## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

In [1]:
!pip install pymorphy2


import random
from gensim import models
from collections import Counter
import re
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import  LogisticRegression
from sklearn import model_selection
import numpy as np
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
lines = list(open('./news_train.txt', 'r', encoding='utf-8'))
random.shuffle(lines)

In [2]:
Counter([line.split('\t')[0] for line in lines[:15000]])

Counter({'business': 554,
         'culture': 2053,
         'economics': 2080,
         'forces': 1225,
         'life': 2033,
         'media': 2111,
         'science': 2156,
         'sport': 2215,
         'style': 284,
         'travel': 289})

In [3]:
training_topics = []
training_text = []

for line in lines:
    lines_split = line.split('\t')
    training_topics.append(lines_split[0])
    training_titles = lines_split[1] 
    training_sentences = lines_split[2]
    training_text.append(training_titles + ' ' + training_sentences)

In [4]:
morph = pymorphy2.MorphAnalyzer()

In [5]:
words = []
for item in training_text:
    words.append(re.sub('[^а-я ]', '', item.replace('.', ' ').lower()).split())

Посмотрим, что мы имеем:

In [6]:
training_text[0]

'Емельяненко и Мальдонадо рассудит вице-президент Союза ММА России Главным судьей боя между Федором Емельяненко и Фабио Мальдонадо станет вице-президент Союза ММА России, председатель Всероссийской коллегии судей и Главный судья Всемирной Ассоциации WMMAA Радмир Габдуллин. Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру».«Я очень рад, что именно судьи Союза ММА России будут работать на этом турнире, где выступят много знаковых бойцов. Для меня лично большая честь быть главным арбитром, я знаю Федора не только как руководителя, но и как хорошего человека, достойного друга и наставника, как пример, на который можно равняться и за которым можно идти», — заявил Габдуллин.19 апреля Мальдонадо заявил о намерении нокаутировать Емельяненко. В тот же день Емельяненко заявил, что хочет встретиться с действующим обладателем чемпионского пояса Абсолютного бойцовского чемпионата (UFC) в тяжелом весе бразильцем Фабрисиу Вердумом.На счету 36-летнего Мальдонадо 31 бой, 22 победы и 

In [7]:
words[0]

['емельяненко',
 'и',
 'мальдонадо',
 'рассудит',
 'вицепрезидент',
 'союза',
 'мма',
 'россии',
 'главным',
 'судьей',
 'боя',
 'между',
 'федором',
 'емельяненко',
 'и',
 'фабио',
 'мальдонадо',
 'станет',
 'вицепрезидент',
 'союза',
 'мма',
 'россии',
 'председатель',
 'всероссийской',
 'коллегии',
 'судей',
 'и',
 'главный',
 'судья',
 'всемирной',
 'ассоциации',
 'радмир',
 'габдуллин',
 'об',
 'этом',
 'сообщается',
 'в',
 'прессрелизе',
 'поступившем',
 'в',
 'редакцию',
 'ленты',
 'ру',
 'я',
 'очень',
 'рад',
 'что',
 'именно',
 'судьи',
 'союза',
 'мма',
 'россии',
 'будут',
 'работать',
 'на',
 'этом',
 'турнире',
 'где',
 'выступят',
 'много',
 'знаковых',
 'бойцов',
 'для',
 'меня',
 'лично',
 'большая',
 'честь',
 'быть',
 'главным',
 'арбитром',
 'я',
 'знаю',
 'федора',
 'не',
 'только',
 'как',
 'руководителя',
 'но',
 'и',
 'как',
 'хорошего',
 'человека',
 'достойного',
 'друга',
 'и',
 'наставника',
 'как',
 'пример',
 'на',
 'который',
 'можно',
 'равняться',
 'и',

In [8]:
for i, line in enumerate(words):
    for j, word in enumerate(line):
        p = morph.parse(word)[0]
        words[i][j] = p.normal_form

In [9]:
words[0]

['емельяненко',
 'и',
 'мальдонадо',
 'рассудить',
 'вицепрезидент',
 'союз',
 'мма',
 'россия',
 'главный',
 'судья',
 'бой',
 'между',
 'фёдор',
 'емельяненко',
 'и',
 'фабио',
 'мальдонадо',
 'стать',
 'вицепрезидент',
 'союз',
 'мма',
 'россия',
 'председатель',
 'всероссийский',
 'коллегия',
 'судья',
 'и',
 'главный',
 'судья',
 'всемирный',
 'ассоциация',
 'радмир',
 'габдуллин',
 'о',
 'это',
 'сообщаться',
 'в',
 'прессрелиз',
 'поступить',
 'в',
 'редакция',
 'лента',
 'ру',
 'я',
 'очень',
 'рад',
 'что',
 'именно',
 'судья',
 'союз',
 'мма',
 'россия',
 'быть',
 'работать',
 'на',
 'это',
 'турнир',
 'где',
 'выступить',
 'много',
 'знаковый',
 'боец',
 'для',
 'я',
 'лично',
 'больший',
 'честь',
 'быть',
 'главный',
 'арбитр',
 'я',
 'знать',
 'фёдор',
 'не',
 'только',
 'как',
 'руководитель',
 'но',
 'и',
 'как',
 'хороший',
 'человек',
 'достойный',
 'друг',
 'и',
 'наставник',
 'как',
 'пример',
 'на',
 'который',
 'можно',
 'равняться',
 'и',
 'за',
 'который',
 'мож

In [10]:
model = models.Word2Vec(min_count = 1, alpha = 0.01, min_alpha = 0.0001, workers = 3)
model.build_vocab(words)
model.train(words, total_examples = model.corpus_count, epochs = 50)
model.wv.most_similar('спорт')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('мутко', 0.5922874212265015),
 ('самбо', 0.5552021265029907),
 ('спартакиада', 0.5309680700302124),
 ('баскетбол', 0.5034136176109314),
 ('рспорт', 0.5031712055206299),
 ('культура', 0.49950510263442993),
 ('виталий', 0.49584901332855225),
 ('молодёжь', 0.4945419430732727),
 ('спортивный', 0.48698872327804565),
 ('футбол', 0.48629340529441833)]

In [11]:
vector = TfidfVectorizer(preprocessor=' '.join) 
X_train = vector.fit_transform(words) 

In [12]:
classifier = LogisticRegression(max_iter = 10000)
classifier.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [13]:
parameters_grid = {'C': [0.0001, 0.001, 0.01, 0.005, 0.1, 1]}

In [14]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid)

In [15]:
d = dict(zip(list(set(training_topics)), range(len(training_topics))))
y_train = [d[key] for key in training_topics]
y_train[:5]

[2, 7, 1, 2, 2]

In [16]:
%%time
grid_cv.fit(X_train, y_train)

CPU times: user 4min 14s, sys: 2min 37s, total: 6min 52s
Wall time: 3min 33s


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=10000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.005, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [17]:
print(grid_cv.best_params_)
grid_cv.best_estimator_

{'C': 1}


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
lines = list(open('./news_test.txt', 'r', encoding='utf-8'))

In [19]:
testing_topics = []
testing_text = []

for line in lines:
    lines_split = line.split('\t')
    testing_topics.append(lines_split[0])
    testing_titles = lines_split[1]
    testing_sentences = lines_split[2]
    testing_text.append(testing_titles + testing_sentences)
words_test = []
for item in testing_text:
    words_test.append(re.sub('[^а-я ]', '', item.replace('.', ' ').lower()).split())
for i, line in enumerate(words_test):
    for j, word in enumerate(line):
        p = morph.parse(word)[0]
        words_test[i][j] = p.normal_form

X_test = vector.transform(words_test) 

y_test = [d[key] for key in testing_topics]
y_test[:5]

[1, 9, 9, 0, 1]

In [20]:
y_pred = grid_cv.best_estimator_.predict(X_test)
classification_report(y_test, y_pred, target_names = d.keys())

'              precision    recall  f1-score   support\n\n    business       0.72      0.32      0.45        90\n     culture       0.92      0.92      0.92       426\n       sport       0.97      0.97      0.97       423\n   economics       0.85      0.92      0.88       426\n      travel       0.94      0.63      0.76        54\n       style       0.88      0.71      0.79        52\n      forces       0.79      0.88      0.84       245\n     science       0.90      0.85      0.87       466\n        life       0.83      0.90      0.86       415\n       media       0.83      0.87      0.85       403\n\n    accuracy                           0.87      3000\n   macro avg       0.87      0.80      0.82      3000\nweighted avg       0.87      0.87      0.87      3000\n'

In [21]:
classifier = MultinomialNB() 
classifier.get_params().keys()  

dict_keys(['alpha', 'class_prior', 'fit_prior'])

In [22]:
parameters_grid = {'alpha': [0, 0.01, 0.1, 1]}

In [23]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid)

In [24]:
%%time
grid_cv.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


CPU times: user 1.23 s, sys: 35.8 ms, total: 1.27 s
Wall time: 1.27 s


GridSearchCV(cv=None, error_score=nan,
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0, 0.01, 0.1, 1]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [25]:
print(grid_cv.best_params_)
grid_cv.best_estimator_

{'alpha': 0.01}


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [26]:
y_pred = grid_cv.best_estimator_.predict(X_test)
classification_report(y_test, y_pred, target_names = d.keys())

'              precision    recall  f1-score   support\n\n    business       0.59      0.40      0.48        90\n     culture       0.91      0.92      0.92       426\n       sport       0.98      0.96      0.97       423\n   economics       0.85      0.89      0.87       426\n      travel       0.90      0.67      0.77        54\n       style       0.91      0.83      0.87        52\n      forces       0.75      0.87      0.80       245\n     science       0.91      0.76      0.83       466\n        life       0.81      0.87      0.84       415\n       media       0.76      0.82      0.79       403\n\n    accuracy                           0.85      3000\n   macro avg       0.84      0.80      0.81      3000\nweighted avg       0.85      0.85      0.85      3000\n'